In [14]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [16]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yV6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D"
]

papers = [
    "metagpt.pdf",
    "swebench.pdf",
    "selfrag.pdf",
    "zipformer.pdf",
    "vr_mcl.pdf"
]

In [17]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: metagpt.pdf
Getting tools for paper: swebench.pdf
Getting tools for paper: selfrag.pdf
Getting tools for paper: zipformer.pdf
Getting tools for paper: vr_mcl.pdf


In [18]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [19]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [21]:
response = agent.query(
    "What is the architecture of MetaGPT and how does it differ from traditional LLM agents?"
)
print(str(response))

Added user message to memory: What is the architecture of MetaGPT and how does it differ from traditional LLM agents?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "The architecture of MetaGPT consists of a large transformer-based language model that is pre-trained on a diverse range of internet text. It differs from traditional LLM agents in its ability to generate human-like text by leveraging meta-learning techniques to adapt quickly to new tasks with few examples. This meta-learning approach allows MetaGPT to generalize better across tasks and domains compared to traditional LLM agents that require extensive fine-tuning for each specific task."}
=== Function Output ===
MetaGPT's architecture involves a transformer-based language model that has undergone pre-training on a wide variety of internet text. Its unique feature is its capability to produce human-like text by utilizing meta-learning techniques for rapid adaptation to new tasks with min